In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, Embedding, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import RMSprop

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import EarlyStopping

In [2]:
full_df = pd.read_csv('data/kokil dec 6 reprepare/conf_good_agg_withpc.csv')

In [3]:
full_df.head()

,HITId,Input.sentence_id,Input.convo_id,Input.train_test_val,Input.msg_id,Input.timestamp,Input.full_text,Input.speaker,Input.reply_to,Input.speaker_intention,...,Answer.3rapport.yes_pc_agree,Answer.4shareinformation.yes_pc_agree,Answer.1gamemove.yes_label,Answer.2reasoning.yes_label,Answer.3a_apologies.yes_label,Answer.3a_compliment.yes_label,Answer.3a_personalthoughts.yes_label,Answer.3a_reassurance.yes_label,Answer.3rapport.yes_label,Answer.4shareinformation.yes_label
0,301KG0KX9CLR06T6MC6UVPAHBC92HU,22056,Game7-turkey-austria,Train,Game7-turkey-austria-9,197,Im moving my fleet to Alb not for Greece but f...,austria-Game7,Game7-turkey-austria-8,Truth,...,1.00,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,301KG0KX9CLR06T6MC6UVPAHBCAH2A,6906,Game11-austria-italy,Validation,Game11-austria-italy-5,45,"And yes I would like peace on our front, I cou...",austria-Game11,Game11-austria-italy-4,Truth,...,1.00,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,301KG0KX9CLR06T6MC6UVPAHBCC2HX,3066,Game1-england-germany,Train,Game1-england-germany-271,1468,"okay...well, as the person who has ever seen a...",germany-Game1,Game1-england-germany-270,Truth,...,1.00,0.666667,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,301KG0KX9CLR06T6MC6UVPAHBCCH2C,24093,Game9-italy-germany,Train,Game9-italy-germany-70,1460,I think the best thing we can do to keep the a...,germany-Game9,Game9-italy-germany-69,Truth,...,0.75,0.750000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,301KG0KX9CLR06T6MC6UVPAHBCD2HY,1591,Game1-england-italy,Train,Game1-england-italy-273,1809,We'll see if I can keep it friendly.,england-Game1,Game1-england-italy-272,Truth,...,0.75,0.750000,1.0,1.0,1.0,1.0,NaN,NaN,1.0,1.0


In [4]:
full_df = full_df.dropna() # dataset contains NaN values, dropping NaNs here

X = full_df['Input.full_text']
y = full_df['Answer.1gamemove.yes_label']

le = LabelEncoder() # this can convert our categories into labels, make sure you don't have NaNs or Nulls in your data first
y = le.fit_transform(y)
print(y.shape)

# we reshape 
y = y.reshape(-1,1) # the -1 allows it to have whatever number went in there
print(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(3295,)
(3295, 1)
(2636,)
(2636, 1)
(659,)
(659, 1)


In [5]:
max_words = 1000
max_len = 50

tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)

sequences = tok.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(sequences,maxlen=max_len)
# X_train = sequence.pad_sequences(sequences)

In [6]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [7]:
early_stop = EarlyStopping(monitor='val_loss',min_delta=0.00001)

In [8]:
Inp = Input(name='inputs',shape=[max_len])
x = Embedding(max_words,50,input_length=max_len)(Inp)
x = LSTM(64,name='LSTM_01')(x)
x = Dropout(0.5,name='Dropout')(x)
x = Dense(128,activation='relu',name='Dense_01')(x)
# x = Dropout(0.5,name='Dropout')(x)
out = Dense(1,activation='sigmoid', name='output')(x)

In [9]:
model = Model(inputs=Inp,outputs=out)

In [10]:
# model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['acc',f1_m,precision_m, recall_m])

In [11]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 50)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 50, 50)            50000     
_________________________________________________________________
LSTM_01 (LSTM)               (None, 64)                29440     
_________________________________________________________________
Dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
Dense_01 (Dense)             (None, 128)               8320      
_________________________________________________________________
output (Dense)               (None, 1)                 129       
Total params: 87,889
Trainable params: 87,889
Non-trainable params: 0
__________________________________________________

In [12]:
model.fit(X_train,y_train,
          batch_size=128,
          epochs=15,
          validation_split=0.2,
          callbacks=[early_stop])

# model.fit(X_train,y_train,
#           batch_size=128,
#           epochs=15,
#           validation_split=0.2)

Epoch 1/15
17/17 [==============================] - 0s 25ms/step - loss: 0.4661 - acc: 0.8615 - f1_m: 0.9256 - precision_m: 0.8716 - recall_m: 0.9886 - val_loss: 0.3382 - val_acc: 0.8939 - val_f1_m: 0.9536 - val_precision_m: 0.9125 - val_recall_m: 1.0000
Epoch 2/15
17/17 [==============================] - 0s 8ms/step - loss: 0.3919 - acc: 0.8700 - f1_m: 0.9316 - precision_m: 0.8725 - recall_m: 1.0000 - val_loss: 0.3413 - val_acc: 0.8939 - val_f1_m: 0.9536 - val_precision_m: 0.9125 - val_recall_m: 1.0000


In [8]:
model = Sequential(name="CNN_with_embeddings")
model.add(Embedding(max_words, 50, input_length=max_len))
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer= 'adam',
              metrics=['acc',f1_m,precision_m, recall_m])

model.summary()

Model: "CNN_with_embeddings"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            50000     
_________________________________________________________________
conv1d (Conv1D)              (None, 43, 32)            12832     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 21, 32)            0         
_________________________________________________________________
flatten (Flatten)            (None, 672)               0         
_________________________________________________________________
dropout (Dropout)            (None, 672)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                6730      
_________________________________________________________________
dense_1 (Dense)              (None, 1)         

In [9]:
early_stop = EarlyStopping(monitor='val_loss',min_delta=0.00001)

In [10]:
model.fit(X_train,y_train,
          batch_size=128,
          epochs=15,
          validation_split=0.2,
          callbacks=[early_stop])

Epoch 1/15
17/17 [==============================] - 0s 16ms/step - loss: 0.5648 - acc: 0.8306 - f1_m: 0.8953 - precision_m: 0.8704 - recall_m: 0.9486 - val_loss: 0.3951 - val_acc: 0.8693 - val_f1_m: 0.9240 - val_precision_m: 0.8594 - val_recall_m: 1.0000
Epoch 2/15
17/17 [==============================] - 0s 4ms/step - loss: 0.4104 - acc: 0.8672 - f1_m: 0.9281 - precision_m: 0.8661 - recall_m: 1.0000 - val_loss: 0.3931 - val_acc: 0.8693 - val_f1_m: 0.9240 - val_precision_m: 0.8594 - val_recall_m: 1.0000
Epoch 3/15
17/17 [==============================] - 0s 4ms/step - loss: 0.3964 - acc: 0.8672 - f1_m: 0.9286 - precision_m: 0.8672 - recall_m: 1.0000 - val_loss: 0.3921 - val_acc: 0.8693 - val_f1_m: 0.9240 - val_precision_m: 0.8594 - val_recall_m: 1.0000
Epoch 4/15
17/17 [==============================] - 0s 4ms/step - loss: 0.3857 - acc: 0.8672 - f1_m: 0.9287 - precision_m: 0.8672 - recall_m: 1.0000 - val_loss: 0.3932 - val_acc: 0.8693 - val_f1_m: 0.9240 - val_precision_m: 0.8594 - val_

In [11]:
test_sequences = tok.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [12]:
model.evaluate(X_test,y_test)

21/21 [==============================] - 0s 2ms/step - loss: 0.3442 - acc: 0.8953 - f1_m: 0.9449 - precision_m: 0.8973 - recall_m: 1.0000


[0.3442312479019165,
 0.8952959179878235,
 0.9449462890625,
 0.8973214030265808,
 1.0]